# Unified Bank Statement Extraction

**Protocol**: Automatic strategy selection based on document characteristics

This notebook uses the `UnifiedBankExtractor` which automatically selects the optimal extraction strategy:

- **2-Turn Balance-Description**: When Balance column is detected (simpler, more reliable)
- **3-Turn Table Extraction**: When no Balance column exists (fallback)

---

## Workflow

```
Turn 0: Image → Header Detection (shared)
    ↓
Python: Pattern matching → Strategy selection
    ↓
Turn 1: Image → Extraction (strategy-specific)
    ↓
Python: Parsing → Filtering → Schema fields
```

## Configuration

- Model configs: `config/model_config.yaml`
- Prompts: `config/bank_statement_prompts.yaml`
- Column patterns: `config/column_patterns.yaml`

In [1]:
# Cell 1: Deterministic setup and imports
# MUST be set before importing torch for deterministic CuBLAS
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import sys
from pathlib import Path

# Get project root (parent of bank_statement/)
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Change working directory to project root
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

# Standard imports
import torch
from PIL import Image
from IPython.display import display

# Reproducibility
from common.reproducibility import set_seed, configure_deterministic_mode
set_seed(42)
configure_deterministic_mode(True)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Working directory: /home/jovyan/nfs_share/tod/LMM_POC
✅ Random seed set to 42 for reproducibility
✅ PyTorch deterministic mode enabled
⚠️  Note: May reduce performance
PyTorch version: 2.5.1
CUDA available: True
CUDA version: 11.8


## Configuration

Select model and image path below.

In [2]:
# Cell 2: Configuration
import yaml

# ============================================================================
# SELECT MODEL
# ============================================================================
# Options: "llama_3_2_vision", "internvl3_8b", "internvl3_2b", "internvl3_5_8b"
MODEL_KEY = "internvl3_5_8b"

# ============================================================================
# SELECT IMAGE
# ============================================================================
IMAGE_PATH = "evaluation_data/bank/minimal/image_003.png"
# IMAGE_PATH = "evaluation_data/bank/minimal/image_008.png"
# IMAGE_PATH = "evaluation_data/bank/minimal/image_009.png"

# ============================================================================
# GROUND TRUTH (optional, for evaluation)
# ============================================================================
GROUND_TRUTH_PATH = "evaluation_data/bank/ground_truth_bank.csv"

# ============================================================================
# Load model config
# ============================================================================
with open("config/model_config.yaml") as f:
    models_config = yaml.safe_load(f)

model_config = models_config["models"][MODEL_KEY]
print(f"Selected model: {model_config['name']}")
print(f"Model type: {model_config['type']}")
print(f"Model path: {model_config['default_path']}")

Selected model: InternVL3.5-8B (bfloat16)
Model type: internvl3
Model path: /home/jovyan/nfs_share/models/InternVL3_5-8B


## Load Model

Model loading is model-type specific. The cell below handles both Llama and InternVL3.

In [3]:
# Cell 3: Load model based on type
import math

MODEL_PATH = model_config["default_path"]
MODEL_TYPE = model_config["type"]

if MODEL_TYPE == "llama":
    # ========================================================================
    # LLAMA MODEL LOADING
    # ========================================================================
    from common.llama_model_loader_robust import load_llama_model_robust
    
    loading_config = model_config.get("loading", {})
    gen_config = model_config.get("generation", {})
    
    print(f"Loading Llama model from: {MODEL_PATH}")
    
    model, processor = load_llama_model_robust(
        model_path=MODEL_PATH,
        use_quantization=loading_config.get("use_quantization", False),
        device_map=loading_config.get("device_map", "auto"),
        max_new_tokens=gen_config.get("max_new_tokens", 4096),
        torch_dtype=loading_config.get("torch_dtype", "bfloat16"),
        low_cpu_mem_usage=loading_config.get("low_cpu_mem_usage", True),
        verbose=True
    )
    
    # Tie weights if configured
    if loading_config.get("tie_weights", True):
        try:
            model.tie_weights()
            print("Model weights tied")
        except Exception as e:
            print(f"tie_weights() warning: {e}")
    
    tokenizer = None  # Llama uses processor
    model_dtype = torch.bfloat16

else:
    # ========================================================================
    # INTERNVL3 MODEL LOADING
    # ========================================================================
    from transformers import AutoModel, AutoTokenizer, AutoConfig, BitsAndBytesConfig
    
    loading_config = model_config.get("loading", {})
    
    print(f"Loading InternVL3 model from: {MODEL_PATH}")
    
    def split_model(model_path):
        """Official InternVL3 multi-GPU device mapping."""
        device_map = {}
        world_size = torch.cuda.device_count()
        config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
        num_layers = config.llm_config.num_hidden_layers
        
        num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
        num_layers_per_gpu = [num_layers_per_gpu] * world_size
        num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
        
        layer_cnt = 0
        for i, num_layer in enumerate(num_layers_per_gpu):
            for _ in range(num_layer):
                device_map[f'language_model.model.layers.{layer_cnt}'] = i
                layer_cnt += 1
        
        device_map['vision_model'] = 0
        device_map['mlp1'] = 0
        device_map['language_model.model.tok_embeddings'] = 0
        device_map['language_model.model.embed_tokens'] = 0
        device_map['language_model.output'] = 0
        device_map['language_model.model.norm'] = 0
        device_map['language_model.model.rotary_emb'] = 0
        device_map['language_model.lm_head'] = 0
        device_map[f'language_model.model.layers.{num_layers - 1}'] = 0
        
        return device_map
    
    world_size = torch.cuda.device_count()
    print(f"Detected {world_size} GPU(s)")
    
    if world_size > 1:
        print("Using multi-GPU bfloat16 mode")
        device_map = split_model(MODEL_PATH)
        model = AutoModel.from_pretrained(
            MODEL_PATH,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=loading_config.get("low_cpu_mem_usage", True),
            use_flash_attn=loading_config.get("use_flash_attn", False),
            trust_remote_code=loading_config.get("trust_remote_code", True),
            device_map=device_map,
        ).eval()
        model_dtype = torch.bfloat16
    else:
        print("Using single-GPU 8-bit quantization mode")
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_enable_fp32_cpu_offload=False
        )
        model = AutoModel.from_pretrained(
            MODEL_PATH,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=loading_config.get("low_cpu_mem_usage", True),
            use_flash_attn=loading_config.get("use_flash_attn", False),
            trust_remote_code=loading_config.get("trust_remote_code", True),
            quantization_config=quantization_config,
            device_map={"":0},
        ).eval()
        model_dtype = torch.float16
    
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_PATH, 
        trust_remote_code=True, 
        use_fast=False
    )
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    processor = None  # InternVL3 doesn't use processor

print(f"\nModel loaded successfully!")
print(f"Model dtype: {model_dtype}")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading InternVL3 model from: /home/jovyan/nfs_share/models/InternVL3_5-8B
Detected 1 GPU(s)
Using single-GPU 8-bit quantization mode
FlashAttention2 is not installed.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

## Load Image

In [ ]:
# Cell 4: Load image
print(f"Loading image: {IMAGE_PATH}")
image = Image.open(IMAGE_PATH).convert("RGB")
print(f"Image size: {image.size}")
display(image)

## Create Extractor and Extract

The `UnifiedBankExtractor` automatically:
1. Detects column headers (Turn 0)
2. Maps headers to semantic types
3. Selects optimal strategy based on Balance column presence
4. Executes extraction
5. Parses and filters results

In [ ]:
# Cell 5: Create extractor and run extraction
from common.unified_bank_extractor import UnifiedBankExtractor

# Create extractor
extractor = UnifiedBankExtractor(
    model=model,
    tokenizer=tokenizer,
    processor=processor,
    model_type=MODEL_TYPE,
    model_dtype=model_dtype,
)

print("="*60)
print("UNIFIED BANK STATEMENT EXTRACTION")
print("="*60)

# Run extraction
result = extractor.extract(image)

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)

## View Results

In [ ]:
# Cell 6: Display extraction results
print("="*60)
print("EXTRACTION METADATA")
print("="*60)
print(f"Strategy used: {result.strategy_used}")
print(f"Turns executed: {result.turns_executed}")
print(f"Headers detected: {result.headers_detected}")
if result.column_mapping:
    print(f"\nColumn mapping:")
    print(f"  Date: {result.column_mapping.date}")
    print(f"  Description: {result.column_mapping.description}")
    print(f"  Debit: {result.column_mapping.debit}")
    print(f"  Credit: {result.column_mapping.credit}")
    print(f"  Balance: {result.column_mapping.balance}")

print("\n" + "="*60)
print("EXTRACTED SCHEMA FIELDS")
print("="*60)
schema = result.to_schema_dict()
for field, value in schema.items():
    display_value = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
    print(f"\n{field}:")
    print(f"  {display_value}")

print("\n" + "="*60)
print("TRANSACTION SUMMARY")
print("="*60)
print(f"Total debit transactions: {len(result.transaction_dates)}")
print(f"Date range: {result.statement_date_range}")

In [ ]:
# Cell 7: Display individual transactions
print("="*60)
print("DEBIT TRANSACTIONS")
print("="*60)

for i, (date, desc, amount) in enumerate(zip(
    result.transaction_dates,
    result.line_item_descriptions,
    result.transaction_amounts_paid
), 1):
    print(f"{i}. [{date}] {desc[:50]}{'...' if len(desc) > 50 else ''} - {amount}")

## Evaluation (Optional)

Compare extracted results against ground truth.

In [ ]:
# Cell 8: Load ground truth and evaluate
import pandas as pd
from pathlib import Path

# Import evaluation metrics
from common.evaluation_metrics import (
    calculate_field_accuracy_f1,
    load_ground_truth,
)

# Bank statement fields to evaluate
BANK_STATEMENT_FIELDS = [
    "DOCUMENT_TYPE",
    "STATEMENT_DATE_RANGE",
    "TRANSACTION_DATES",
    "LINE_ITEM_DESCRIPTIONS",
    "TRANSACTION_AMOUNTS_PAID",
]

# Load ground truth
print("Loading ground truth...")
try:
    ground_truth_map = load_ground_truth(GROUND_TRUTH_PATH, verbose=True)
    
    # Get image filename
    image_filename = Path(IMAGE_PATH).name
    print(f"Looking up: {image_filename}")
    
    if image_filename in ground_truth_map:
        gt_data = ground_truth_map[image_filename]
        print("Ground truth found!")
        
        # Calculate F1 scores
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        
        total_f1 = 0.0
        for field in BANK_STATEMENT_FIELDS:
            extracted = schema.get(field, "NOT_FOUND")
            ground_truth = gt_data.get(field, "NOT_FOUND")
            
            if pd.isna(ground_truth):
                ground_truth = "NOT_FOUND"
            
            result_eval = calculate_field_accuracy_f1(extracted, ground_truth, field)
            f1 = result_eval.get("f1_score", 0.0)
            total_f1 += f1
            
            status = "OK" if f1 == 1.0 else ("PARTIAL" if f1 >= 0.5 else "FAIL")
            print(f"{status:7} {field:30} F1: {f1:.1%}")
        
        overall_f1 = total_f1 / len(BANK_STATEMENT_FIELDS)
        print("\n" + "-"*60)
        print(f"OVERALL F1 SCORE: {overall_f1:.1%}")
        print("="*60)
    else:
        print(f"No ground truth found for: {image_filename}")
        
except FileNotFoundError:
    print(f"Ground truth file not found: {GROUND_TRUTH_PATH}")
    print("Skipping evaluation.")

## View Raw Responses (Debug)

In [ ]:
# Cell 9: View raw LLM responses (for debugging)
print("="*60)
print("RAW TURN 0 RESPONSE (Header Detection)")
print("="*60)
print(result.raw_responses.get("turn0", "N/A"))

print("\n" + "="*60)
print("RAW TURN 1 RESPONSE (Extraction)")
print("="*60)
print(result.raw_responses.get("turn1", "N/A")[:10000])
if len(result.raw_responses.get("turn1", "")) > 10000:
    print("... [truncated]")

## Summary

In [ ]:
# Cell 10: Final summary
print("\n" + "="*60)
print("EXTRACTION SUMMARY")
print("="*60)
print(f"\nModel: {model_config['name']}")
print(f"Image: {IMAGE_PATH}")
print(f"Strategy: {result.strategy_used}")
print(f"Headers: {len(result.headers_detected)} detected")
print(f"Balance column: {result.column_mapping.balance if result.column_mapping else 'N/A'}")
print(f"Transactions extracted: {len(result.transaction_dates)} debits")
print(f"Date range: {result.statement_date_range}")
print(f"\nPipeline complete!")